## <font color=red> You should not import any new libraries. Your code should run with python=3.x</font>
## <font color=red> Please don't rename this .ipynb file.</font><br>
- Your solutions will be auto-graded. Hence we request you to follow the instructions.
- Modify the code only between 
```
## TODO
## END TODO
```
- In addition to above changes, you can play with arguments to the functions for generating plots
- We will run the auto grading scripts with private test data

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from scipy import linalg

c:\users\aakri\appdata\local\programs\python\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\aakri\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
c:\users\aakri\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\aakri\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


## Please make sure that your code works with loading data from relative path only

i.e. ```pd.read_csv('./data/multi_var_lasso.csv')``` should not throw an error when we run the auto-grading scripts

In [2]:
data_multi = pd.read_csv('./data/multi_var_lasso.csv')
cols = [f"x_gt_{idx}" for idx in range(1, 6)]
X_multi = np.array(data_multi[cols])
Y_multi = np.array(data_multi['y_gt'])

In [3]:
data_multi_class = pd.read_csv('./data/3_class_perceptron.csv')
cols = [f"x_gt_{idx}" for idx in range(1, 3)]
X_multi_class = np.array(data_multi_class[cols])
Y_multi_class = np.array(data_multi_class['y_gt'])

In [4]:
def mse_multi_var(X, Y, w, b):
    '''
    Compute mean squared error between predictions and true y values

    Args:
    X - numpy array of shape (n_samples, 1)
    Y - numpy array of shape (n_samples, 1)
    w - a float
    b - a float
    '''

    ## TODO
    
    y_pred = np.dot(X, w) + b
    mse = np.mean(np.square(y_pred - Y))

    ## END TODO

    return mse

In [5]:
def mse_regularized(X, Y, w, b, lamda):
    '''
    Compute mean squared error between predictions and true y values

    Args:
    X - numpy array of shape (n_samples, 1)
    Y - numpy array of shape (n_samples, 1)
    w - a float
    b - a float
    '''
    ## TODO
    
    n_samples = len(Y)
    y_pred = np.dot(X, w) + b
    mse = np.mean(np.square(y_pred - Y)) + lamda*np.sum(np.abs(w))/n_samples

    ## END TODO

    return mse

## Plot Graphs

- This function plots the ground truth curve in <font color=green>green</font> and the predicted function in <font color=red>red</font>

In [6]:
def plot_curves(w, b, x, y):
  x_gt = np.linspace(-1, 2, 50)
  y_gt = 1 - 3 * x_gt - 2 * x_gt ** 2 + 2.5 * x_gt ** 3
  # print(x_gt.shape,y_gt.shape)
  if len(w) == 1:
    y_fit = w * x_gt + b
  elif len(w) == 5:
    x_fit = x_gt
    for pow in range(2, 4):
      x_fit = np.vstack([x_fit, np.power(x_gt, pow)])

    x_fit = np.vstack([x_fit, np.power(x_gt, 2)])
    x_fit = np.vstack([x_fit, np.power(x_gt, 1)])
    
    y_fit = np.dot(w, x_fit) + b
  else:
    assert False, 'Pass a valid w'
  plt.plot(x_gt, y_gt, color="green", label='1 - 3 * x - 2 * x ** 2 + 2.5 * x ** 3')
  plt.plot(x_gt, y_fit, color='red', label="Fitted Function y = w.Tx + b")
  if len(x.shape) == 1:
    x_plot = np.vstack([x, np.ones(len(x))]).T
    # print (x_plot.shape, y.shape)
  else:
    x_plot = x
  plt.scatter(x_plot[:,0],y)
  plt.legend()
  plt.title("Ground Truth Function")
  plt.show()

In [7]:
def split_data(X, Y, train_ratio=0.6):
    '''
    Split data into train and validation sets
    The first floor(train_ratio*n_sample) samples form the train set
    and the remaining the test set

    Args:
    X - numpy array of shape (n_samples, n_features)
    Y - numpy array of shape (n_samples, 1)
    train_ratio - fraction of samples to be used as training data

    Returns:
    X_train, Y_train, X_val, Y_val
    '''
    ## TODO
    
    split_val = int(len(Y)*train_ratio)
    rng = np.random.default_rng()
    arr = np.arange(len(Y))
    rng.shuffle(arr)
    X = X[arr]
    Y = Y[arr]
    X_train = X[:split_val]
    Y_train = Y[:split_val]
    X_val = X[split_val:]
    Y_val = Y[split_val:]
    
    ## END TODO

    return X_train, Y_train, X_val, Y_val

# Lasso Regression

In [8]:
X_train, Y_train, X_val, Y_val = split_data(X_multi, Y_multi, train_ratio=0.6)

def ista(X_train, Y_train, X_val, Y_val, epochs=100, lr=1e-3, lamda = 1):
    '''
    Perform multi variable lasso regression using ISTA

    Args:
    X_train - numpy array of shape (n_samples_train, 5)
    Y_train - numpy array of shape (n_samples_train, 1)
    X_val - numpy array of shape (n_samples_val, 5)
    Y_val - numpy array of shape (n_samples_val, 1)
    epochs - number of gradient descent steps
    lr - learnig rate
    lamda - regularization_weight
    '''

    w = np.zeros(X_train.shape[1]) #np.random.randn(X_train.shape[1])
    b = 0
    ## TODO
    
    def soft_threshold(alpha, x):
        if x > alpha:
            return x - alpha
        elif x < -alpha:
            return x + alpha
        else:
            return 0
    
    n = len(Y_train)
    m = X_train.shape[1]
    for i in range(epochs):
        
        y_pred = np.dot(X_train, w) + b        
        dw_ols = 2*lr*(np.dot(X_train.T, (y_pred - Y_train)))/n  
        w_soft = np.zeros(w.shape)
        for j in range(m):
            w_soft[j] = soft_threshold(lr*lamda, w[j])
            
        w = w_soft - dw_ols
        
        
        db = 2*lr*np.sum(y_pred - Y_train)/n        
        b = b - db

    ## END TODO
    
    mse_train = mse_regularized(X_train, Y_train, w, b, lamda)
    mse_val = mse_regularized(X_val, Y_val, w, b, lamda)
    print(f'Validation loss if {mse_val}')
    print(f'Training Loss loss if {mse_train}')
    print(w.shape,b,X_train.shape,Y_train.shape)
    plot_curves(list(w), b, X_train, Y_train)
    return w, b

ista(X_train, Y_train, X_val, Y_val)

Validation loss if 4.433381122004656
Training Loss loss if 1.9432436980943835
(5,) 0.07523276845560356 (30, 5) (30,)


(array([-0.00030869,  0.05886595,  0.20743301,  0.05886595, -0.00030869]),
 0.07523276845560356)

# Ridge Regression

In [9]:
def multivar_reg_closedform(X_train, Y_train, X_val, Y_val, lamda=0.5):

    '''
    Perform L2 regularized multi variable least squares regression using 
    closed form update rules

    Args:
    X_train - numpy array of shape (n_samples_train, 5)
    Y_train - numpy array of shape (n_samples_train, 1)
    X_val - numpy array of shape (n_samples_val, 5)
    Y_val - numpy array of shape (n_samples_val, 1)
    lambda - regularization weight
    '''

    w = np.zeros(X_train.shape[1])
    b = 0

    ## TODO     #use closed-form solution from previous assignment

    n = len(Y_train)
    w = np.array(w)
    X = np.hstack((X_train, np.ones((X_train.shape[0], 1), dtype=X_train.dtype)))
    w_b = np.dot(np.linalg.inv(np.dot(X.T, X) + lamda*np.identity(X.shape[1])), np.dot(X.T, Y_train)) 
    
    w = w_b[:-1]
    b = w_b[-1]

    ## END TODO

    mse_train = mse_multi_var(X_train, Y_train, w, b)
    mse_val = mse_multi_var(X_val, Y_val, w, b)
    print(f'Validation loss if {mse_val}')
    print(f'Training Loss loss if {mse_train}')
    plot_curves(list(w), b, X_train, Y_train)
    
    return w, b

multivar_reg_closedform(X_train, Y_train, X_val, Y_val)

Validation loss if 0.04850963567340867
Training Loss loss if 0.026713724370033713


(array([-1.37585614, -0.75415854,  2.15734326, -0.75415854, -1.37585614]),
 0.8143561697583741)

# Bias-Variance Tradeoff

In [10]:
#
def ridge(X_train, Y_train, X_test,epochs=20, lr=2e-1, lamda = 1):

    w = 0.6
    b = 2
    n = float(len(X_train)) # Number of elements in X

    # Performing Gradient Descent 

    ## TODO
    
    for i in range(epochs):
        
        y_pred = w*X_train + b        
        dw = 2*(np.dot(X_train.T, (y_pred - Y_train)))/n + 2*lamda*w
        db = 2*np.sum(y_pred - Y_train)/n
        w -= lr*dw
        b -= lr*db

    ### END TODO

    Y_pred = w*X_test+b
    return Y_pred

def lasso(X_train, Y_train, X_test,epochs=20, lr=2e-1, lamda = 1):
    w = 0.6
    b = 2
    n = float(len(X_train)) # Number of elements in X

    # Performing Gradient Descent 
    ##TODO
    
    def soft_threshold(alpha, x):
        if x > alpha:
            return x - alpha
        elif x < -alpha:
            return x + alpha
        else:
            return 0
    
    for i in range(epochs):
        
        y_pred = w*X_train + b        
        dw_ols = 2*lr*(np.dot(X_train.T, (y_pred - Y_train)))/n  
        w_soft = soft_threshold(lr*lamda, w)
            
        w = w_soft - dw_ols
        
        
        db = 2*lr*np.sum(y_pred - Y_train)/n        
        b = b - db

    ##END TODO

    Y_pred = w*X_test+b
    return Y_pred

def ols(X_train, Y_train, X_test,epochs=20, lr=2e-1):

    w = 0.6
    b = 2
    n = float(len(X_train)) # Number of elements in X

    # Performing Gradient Descent 
    ## TODO
    
    for i in range(epochs):
        
        y_pred = w*X_train + b
        dw = 2*(np.dot((y_pred - Y_train).T, X_train))/n
        db = 2*np.sum(y_pred - Y_train)/n
        w -= lr*dw
        b -= lr*db

    ### END TODO

    Y_pred = w*X_test+b
    return Y_pred

In [11]:
# We fit multiple lines onto our linear model defined by y = c + mx + error.
num_lines = 10
#size of Dataset, len(Dataset) for 1 line
n = 500
#weights (slope/intercept)
c = 3
m = 5

In [12]:
def gen_data():
    '''
    We sample n, X from uniform distribution and error from zero mean normal distribtion, we find Y using mx+c+e
    We do the above for num_lines number of time, to fit different lines for each models(lasso, ols, ridge), doing so will
    be helpfull in calculating Expected value of learned esitmator of all n*num_lines dataset

    DO NOT CHANGE this function.
    '''

    w0 = c
    w1 = m
    data = np.zeros(shape=(num_lines,n,2))
    for i in range(num_lines):
        x = np.random.uniform(-2, 2, size=(n, 1))
        e = np.random.normal(0, 8, size=(n, 1))
        y = w0 + w1 * x + e
        x_y = np.concatenate((x, y), axis=1)
        data[i,:,:] = x_y
    return np.array(data)

In [13]:
def gen_bais_variance(Y_preds, Y_true):
    ## TODO
    
    mu_pred = np.mean(Y_preds, axis = 0)
    variance = np.mean(np.square(Y_preds - mu_pred), axis=0)
    bias = mu_pred - np.repeat(Y_true, 3)

    ## END TODO
    return bias, variance

In [14]:
def make_prediction(data, X_test, lambda_=0.5):

  y_hat = np.zeros(shape=(num_lines, 3))#store prediction of all model

  for i in range(num_lines):
      X = data[i, :, 0].reshape(n ,1)
      y = data[i, :, 1].reshape(n, 1)

      y_hat[i, 0] = ols(X, y, X_test)

      y_hat[i, 1] = ridge(X, y, X_test, lamda = lambda_)

      y_hat[i, 2] = lasso(X, y, X_test, lamda = lambda_)

  return y_hat

In [15]:
def plot_figure(l,b,v):

  plt.plot(l, b[:, 0], label="OLS")
  plt.plot(l, b[:, 1], label="Ridge")
  plt.plot(l, b[:, 2], label="Lasso")
  plt.xlabel("Regularization coefficient")
  plt.ylabel("Bias")
  plt.title("Bias vs Lambda")
  plt.legend(loc="upper left")
  plt.show()

  plt.plot(l, v[:, 0], label="OLS")
  plt.plot(l, v[:, 1], label="Ridge")
  plt.plot(l, v[:, 2], label="Lasso")
  plt.xlabel("Regularization coefficient")
  plt.ylabel("Variance")
  plt.title("Variance vs Lambda")
  plt.legend(loc="lower left")
  plt.show()



In [16]:
def driver():
  all_lambda = np.linspace(0.0001, 1, 100)
  all_bias = np.zeros(shape=(len(all_lambda), 3))
  all_variance = np.zeros(shape=(len(all_lambda), 3))
  dataset = gen_data()
  X_test = np.array([4]).reshape((1,1))
  for i, l in enumerate(all_lambda):
      Y_hat = make_prediction(dataset, X_test, lambda_=l)
      ##TODO #calculate true mean Y_true
        
      Y_true = m*X_test + c

      ##END TODO
      all_bias[i, :], all_variance[i, :] = gen_bais_variance(Y_hat, Y_true) 
  plot_figure(all_lambda, all_bias, all_variance)

driver()

# Perceptron

In [17]:
X_train_multiclass, Y_train_multiclass, X_val_multiclass, Y_val_multiclass = split_data(X_multi_class, Y_multi_class, train_ratio=0.6)

def perceptron_algo(X_train_multiclass, Y_train_multiclass, X_val_multiclass, Y_val_multiclass,n_class=3, epochs=30):
  W=np.zeros((n_class, len(X_train_multiclass[0]) ))
  ## TODO
    
  for it in range(epochs):
    for i in range(len(Y_train_multiclass)):
      y_pred=np.argmax([np.dot(W[0],X_train_multiclass[i]),np.dot(W[1],X_train_multiclass[i]),
                      np.dot(W[2],X_train_multiclass[i])])
      if y_pred != Y_train_multiclass[i]:
        W[y_pred] -= X_train_multiclass[i]
        W[Y_train_multiclass[i]] += X_train_multiclass[i]
  ##END TODO

  Y_pred=list()
  for i in range(len(Y_val_multiclass)):
      y_pred=np.argmax([np.dot(W[0],X_val_multiclass[i]),np.dot(W[1],X_val_multiclass[i]),np.dot(W[2],X_val_multiclass[i])])
      Y_pred.append(y_pred)

  from sklearn.metrics import classification_report
  target_names = ['class 0', 'class 1', 'class 2']
  print(classification_report(Y_val_multiclass, Y_pred, target_names=target_names))
  return W

perceptron_algo(X_train_multiclass, Y_train_multiclass, X_val_multiclass, Y_val_multiclass)


              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00        15
     class 1       1.00      0.85      0.92        13
     class 2       0.86      1.00      0.92        12

    accuracy                           0.95        40
   macro avg       0.95      0.95      0.95        40
weighted avg       0.96      0.95      0.95        40



array([[ 9.16443934,  1.93727154],
       [-6.78715433,  5.40680978],
       [-2.377285  , -7.34408132]])